In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import matplotlib.font_manager as font_manager
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [8]:
df = pd.read_csv("./data/database_09042024.csv")
df.head(2)

,Code1,Code2,Code3,Date measured,Temperature,Humidity,901.398183,905.331226,909.259927,913.184288,...,1673.98141,1676.942091,1679.898431,1682.85043,1685.798089,1688.741406,1691.680383,1694.615018,1697.545313,1701.44562
0,Dữ lieu do NIR - 2022-06-23,00893_36,Hadanard1_100800_20220623_141053.csv,6/23/2022 14:10,34.29,85.91,0.830749,0.807264,0.816415,0.812683,...,-0.172542,-0.1899,-0.200437,-0.196828,-0.17931,-0.16928,-0.156739,-0.154597,-0.150397,-0.1262
1,Dữ lieu do NIR - 2022-06-23,00893_36,Hadanard1_100800_20220623_141053.csv,6/23/2022 14:10,34.29,85.91,0.830749,0.807264,0.816415,0.812683,...,-0.172542,-0.1899,-0.200437,-0.196828,-0.17931,-0.16928,-0.156739,-0.154597,-0.150397,-0.1262


In [9]:
len(df)

65274

In [10]:
df.drop_duplicates(inplace=True)

In [12]:
len(df)

31087

In [13]:
df.head(2)

,Code1,Code2,Code3,Date measured,Temperature,Humidity,901.398183,905.331226,909.259927,913.184288,...,1673.98141,1676.942091,1679.898431,1682.85043,1685.798089,1688.741406,1691.680383,1694.615018,1697.545313,1701.44562
0,Dữ lieu do NIR - 2022-06-23,00893_36,Hadanard1_100800_20220623_141053.csv,6/23/2022 14:10,34.29,85.91,0.830749,0.807264,0.816415,0.812683,...,-0.172542,-0.189900,-0.200437,-0.196828,-0.179310,-0.169280,-0.156739,-0.154597,-0.150397,-0.126200
2,Dữ lieu do NIR - 2022-06-23,00893_36,Hadanard1_100800_20220623_141110.csv,6/23/2022 14:11,34.48,85.26,0.704169,0.701405,0.692683,0.690105,...,-0.069185,-0.090661,-0.098000,-0.096262,-0.084292,-0.072022,-0.062244,-0.054861,-0.055234,-0.031353


In [14]:
df.to_csv("./data/database_09042024.csv", index=False, encoding='utf-8-sig')  

In [43]:
df = pd.read_csv("./data/database_09042024.csv")
df.head(2)

,Code1,Code2,Code3,Date measured,Temperature,Humidity,901.398183,905.331226,909.259927,913.184288,...,1673.98141,1676.942091,1679.898431,1682.85043,1685.798089,1688.741406,1691.680383,1694.615018,1697.545313,1701.44562
0,Dữ lieu do NIR - 2022-06-23,00893_36,Hadanard1_100800_20220623_141053.csv,6/23/2022 14:10,34.29,85.91,0.830749,0.807264,0.816415,0.812683,...,-0.172542,-0.189900,-0.200437,-0.196828,-0.179310,-0.169280,-0.156739,-0.154597,-0.150397,-0.126200
1,Dữ lieu do NIR - 2022-06-23,00893_36,Hadanard1_100800_20220623_141110.csv,6/23/2022 14:11,34.48,85.26,0.704169,0.701405,0.692683,0.690105,...,-0.069185,-0.090661,-0.098000,-0.096262,-0.084292,-0.072022,-0.062244,-0.054861,-0.055234,-0.031353


In [44]:
df.insert(4, 'ID_machine', np.nan)
df['ID_machine'] = df.apply(lambda x: 2 if x['Code3'][8]=="1" else 1, axis=1)
df.head(1)

,Code1,Code2,Code3,Date measured,ID_machine,Temperature,Humidity,901.398183,905.331226,909.259927,...,1673.98141,1676.942091,1679.898431,1682.85043,1685.798089,1688.741406,1691.680383,1694.615018,1697.545313,1701.44562
0,Dữ lieu do NIR - 2022-06-23,00893_36,Hadanard1_100800_20220623_141053.csv,6/23/2022 14:10,2,34.29,85.91,0.830749,0.807264,0.816415,...,-0.172542,-0.1899,-0.200437,-0.196828,-0.17931,-0.16928,-0.156739,-0.154597,-0.150397,-0.1262


In [59]:
df['extract_date_from_filename'] = df.apply(lambda x: datetime.strptime(x['Code3'][18:-4],"%Y%m%d_%H%M%S")  if x['ID_machine'] == 1 else datetime.strptime(x['Code3'][17:-4],"%Y%m%d_%H%M%S"), axis=1)
df['extract_date_from_foldername'] = df.apply(lambda x: datetime.strptime(x['Code1'][-10:],"%Y-%m-%d") 
                    if pd.notnull(x['Code1'])
                    else (datetime.strptime(x['Code3'][18:-4],"%Y%m%d_%H%M%S")  
                            if x['ID_machine'] == 1 
                            else datetime.strptime(x['Code3'][17:-4],"%Y%m%d_%H%M%S")), axis=1)

In [60]:
df['extract_date_from_filename'] = pd.to_datetime(df["extract_date_from_filename"]).dt.date
df['extract_date_from_foldername'] = pd.to_datetime(df["extract_date_from_foldername"]).dt.date

In [61]:
df['date_fix2'] = df.apply(lambda x: 1 if abs(x['extract_date_from_filename'] - x['extract_date_from_foldername']).days <= 14 else 0, axis=1)

In [62]:
print(len(df[df['date_fix2']==0]))
df[df['date_fix2']==0]['extract_date_from_filename'].unique()

2061


array([datetime.date(2015, 3, 31)], dtype=object)

In [63]:
df['extract_date_from_filename'] = df.apply(lambda x: x['extract_date_from_filename'] if abs(x['extract_date_from_filename'] - x['extract_date_from_foldername']).days <= 14 else x['extract_date_from_foldername'], axis=1)

In [64]:
df.to_csv("data/data_tieu_hat_full.csv", index=False, encoding='utf-8-sig')

# Hóa lý

In [6]:
df_hoaly = pd.read_excel("./data/Hoa_ly_09042024.xlsx", sheet_name="Sheet2")
df_hoaly.drop_duplicates(inplace=True)
df_hoaly = df_hoaly.reset_index(drop=True)
df_hoaly.head(2)

,Sample,Date_HoaLy,DoAm,TroTong,TroKhongTan,Piperine,TinhDau
0,00425_02,2022-01-20 00:00:00,14.575133,3.443350,NaN,4.3154,2.8
1,00423_04,2021-12-02 00:00:00,14.529467,4.355533,0.432867,4.5909,2.6


In [7]:
df_hoaly['Date_HoaLy'] = pd.to_datetime(df_hoaly["Date_HoaLy"]).dt.date
df_hoaly.columns = ['Sample', 'Date_HoaLy', 'DoAm', 'TroTong', 'Peperine', 'TinhDau']

ValueError: Length mismatch: Expected axis has 7 elements, new values have 6 elements

In [8]:
df = pd.read_csv("data/data_tieu_hat_full.csv")
df = df.replace({'02415-07':'02415_07', 
            '02422-09':'02422_09',
            '00389_02':'00398_02',
            '09625_15':'06925_15'})

In [9]:
arr_code_NIR = df['Code2'].unique()
arr_code_hoaly = df_hoaly['Sample'].unique()

In [10]:
filtered_array = [x for x in arr_code_NIR if x not in arr_code_hoaly]
filtered_array

['08198_01',
 '08216_02',
 '08298_06',
 '00898_23',
 '00871_01',
 '00373_08',
 '00401_22',
 '02469_32',
 '00523_01',
 '00534_03']

In [11]:
[x for x in arr_code_hoaly if x not in arr_code_NIR]


['08201_17',
 '00426_01',
 '00373_01',
 '00559_23',
 '02255_24',
 '00506_22',
 '00559_19',
 '08406_21',
 '02415_11']

In [5]:
len(df['Code2'].unique())

151

In [6]:
len(df_hoaly['Sample'].unique())

150

In [87]:
df[df['Code2'] == '02649_34']['extract_date_from_filename'].unique()

array(['11/28/2022', '12/13/2022'], dtype=object)

In [86]:
df_hoaly[df_hoaly['Sample'] == '02649_34']

,Sample,Date_HoaLy,DoAm,TroTong,Peperine,TinhDau
111,02649_34,2022-12-01,11.766200,4.107300,3.831183,3.20000
126,02649_34,2022-12-20,12.512700,4.163100,4.606467,2.10000
180,02649_34,2023-12-12,13.279642,4.044492,NaN,2.39952


In [81]:
df_hoaly[df_hoaly['Sample'].duplicated()]

,Sample,Date_HoaLy,DoAm,TroTong,Peperine,TinhDau
84,07338_19,2022-07-05,11.833700,3.464400,4.313000,2.80000
112,02455_03,2022-12-01,15.256300,3.773700,4.653033,3.60000
126,02649_34,2022-12-20,12.512700,4.163100,4.606467,2.10000
128,02649_23,2022-12-22,12.280500,4.078600,4.402683,2.90000
156,00506_18,2023-09-11,12.477503,4.548414,NaN,2.09958
157,02255_25,2023-11-13,12.873494,4.390763,NaN,2.69946
158,02255_26,2023-11-16,14.694814,4.268711,NaN,4.09918
159,08406_21,2023-11-29,13.206585,3.537438,NaN,3.09938
160,02255_24,2023-12-01,13.016023,4.443201,NaN,3.09938
161,00506_22,2023-12-08,14.673643,4.032633,NaN,3.19936


In [7]:
df['DoAm'] = pd.Series(dtype='int')
df['TroTong'] = pd.Series(dtype='int')
#df['TroKhongTan'] = pd.Series(dtype='int')
df['Peperine'] = pd.Series(dtype='int')
df['TinhDau'] = pd.Series(dtype='int')

In [8]:
df['extract_date_from_filename'] = pd.to_datetime(df['extract_date_from_filename'] ).dt.date

In [157]:
for i in range(len(df_hoaly)):
    try:
        df['DoAm'] = df.apply(lambda x: df_hoaly['DoAm'][i] if ((x['Code2'] == df_hoaly['Sample'][i]) and (abs(x['extract_date_from_filename'] - df_hoaly['Date_HoaLy'][i]).days <= 30)) else x['DoAm'], axis=1)
    except: 
        pass
for i in range(len(df_hoaly)):
    try:
        df['TroTong'] = df.apply(lambda x: df_hoaly['TroTong'][i] if ((x['Code2'] == df_hoaly['Sample'][i]) and (abs(x['extract_date_from_filename'] - df_hoaly['Date_HoaLy'][i]).days <= 30)) else x['TroTong'], axis=1)
    except: 
        pass 
for i in range(len(df_hoaly)):
    try:
        df['TroKhongTan'] = df.apply(lambda x: df_hoaly['TroKhongTan'][i] if ((x['Code2'] == df_hoaly['Sample'][i]) and (abs(x['extract_date_from_filename'] - df_hoaly['Date_HoaLy'][i]).days <= 30)) else x['TroKhongTan'], axis=1)
    except: 
        pass   

for i in range(len(df_hoaly)):
    try:
        df['Peperine'] = df.apply(lambda x: df_hoaly['Peperine'][i] if ((x['Code2'] == df_hoaly['Sample'][i]) and (abs(x['extract_date_from_filename'] - df_hoaly['Date_HoaLy'][i]).days <= 30)) else x['Peperine'], axis=1)
    except: 
        pass

for i in range(len(df_hoaly)):
    try:
        df['TinhDau'] = df.apply(lambda x: df_hoaly['TinhDau'][i] if ((x['Code2'] == df_hoaly['Sample'][i]) and (abs(x['extract_date_from_filename'] - df_hoaly['Date_HoaLy'][i]).days <= 30)) else x['TinhDau'], axis=1)
    except: 
        pass

In [9]:
for i in range(len(df_hoaly)):
    df['DoAm'] = df.apply(lambda x: df_hoaly['DoAm'][i] if ((x['Code2'] == df_hoaly['Sample'][i]) and (abs(x['extract_date_from_filename'] - df_hoaly['Date_HoaLy'][i]).days <= 15)) else x['DoAm'], axis=1)
    df['TroTong'] = df.apply(lambda x: df_hoaly['TroTong'][i] if ((x['Code2'] == df_hoaly['Sample'][i]) and (abs(x['extract_date_from_filename'] - df_hoaly['Date_HoaLy'][i]).days <= 15)) else x['TroTong'], axis=1)
    #df['TroKhongTan'] = df.apply(lambda x: df_hoaly['TroKhongTan'][i] if ((x['Code2'] == df_hoaly['Sample'][i]) and (abs(x['extract_date_from_filename'] - df_hoaly['Date_HoaLy'][i]).days <= 30)) else x['TroKhongTan'], axis=1)
    df['Peperine'] = df.apply(lambda x: df_hoaly['Peperine'][i] if ((x['Code2'] == df_hoaly['Sample'][i]) and (abs(x['extract_date_from_filename'] - df_hoaly['Date_HoaLy'][i]).days <= 15)) else x['Peperine'], axis=1)
    df['TinhDau'] = df.apply(lambda x: df_hoaly['TinhDau'][i] if ((x['Code2'] == df_hoaly['Sample'][i]) and (abs(x['extract_date_from_filename'] - df_hoaly['Date_HoaLy'][i]).days <= 15)) else x['TinhDau'], axis=1)


In [10]:
df.to_csv("./data/data_match1.csv", index=False, encoding="utf-8-sig")

# Peperine

In [44]:
df = pd.read_csv("./data/data_match1.csv")

In [17]:
len(df1['Code2'].unique())

45

In [49]:
df1 = df[pd.notna(df['TinhDau'])]

In [50]:
len(df1)

18358

In [51]:
df1.to_csv("./data/data_NIR_Tinhdau.csv", encoding="utf-8-sig", index=False)

In [23]:
df_hoaly_tinhdau = df_hoaly[pd.notna(df_hoaly['TinhDau'])]

In [24]:
arr_code_NIR = df1['Code2'].unique()
arr_code_hoaly = df_hoaly_tinhdau['Sample'].unique()

In [40]:
i = "08297_05"

print(df[df['Code2'] == i]['extract_date_from_filename'].unique())
print(df1[df1['Code2'] == i]['extract_date_from_filename'].unique())
print(df_hoaly_tinhdau[df_hoaly_tinhdau['Sample'] == i])

['1/11/2022' '2/15/2022' '4/18/2022' '11/9/2021' '11/12/2021' '11/15/2021']
['1/11/2022' '2/15/2022' '11/9/2021' '11/12/2021' '11/15/2021']
      Sample  Date_HoaLy       DoAm   TroTong  TroKhongTan  Piperine  TinhDau
31  08297_05  2022-04-18  12.254656  4.572334          NaN  5.710932      3.6


In [29]:
[x for x in arr_code_NIR if x in arr_code_hoaly]

['08198_20',
 '08297_05',
 '02422_04',
 '02563_02',
 '02455_03',
 '02485_05',
 '02506_01',
 '08198_19',
 '08299_13',
 '08299_14',
 '08299_15',
 '08299_16',
 '00389_01',
 '08299_12',
 '00425_02',
 '00506_05',
 '00401_16',
 '06925_15',
 '00401_17',
 '00401_18',
 '06925_13',
 '02411_06',
 '02417_10',
 '00423_04',
 '08198_02',
 '08198_05',
 '08216_01',
 '08216_03',
 '08297_04',
 '08297_06',
 '08299_07',
 '08299_08',
 '08299_09',
 '08299_10',
 '08299_11']

In [39]:
df1['extract_date_from_filename'].unique()

array(['1/5/2022', '1/11/2022', '1/18/2022', '2/12/2022', '2/15/2022',
       '2/17/2022', '2/28/2022', '3/7/2022', '3/17/2022', '3/24/2022',
       '3/29/2022', '4/4/2022', '5/31/2022', '7/1/2022', '11/7/2022',
       '4/26/2023', '5/5/2023', '5/16/2023', '5/9/2023', '7/14/2023',
       '12/2/2022', '11/9/2021', '11/12/2021', '11/15/2021', '11/16/2021',
       '11/23/2021', '12/2/2021'], dtype=object)

In [45]:
df['extract_date_from_filename'] = df['extract_date_from_filename'].replace({"1/19/2022":'1/18/2022',
                                                                               "2/10/2022":'2/12/2022',
                                                                               "2/18/2022":'2/17/2022',
                                                                                "3/8/2022":'3/7/2022',
                                                                                "3/15/2022":'3/17/2022',
                                                                                "3/18/2022":'3/17/2022',
                                                                                "3/22/2022":'3/24/2022',
                                                                                "12/1/2021":'12/2/2021',
                                                                                "12/4/2021":'12/2/2021',                                                                   
                                                                               })

In [41]:
for i in a:
    print(i)
    print(df[df['Code2'] == i]['extract_date_from_filename'].unique())
    print(df1[df1['Code2'] == i]['extract_date_from_filename'].unique())
    print(df_hoaly_tinhdau[df_hoaly_tinhdau['Sample'] == i])
    print("\n")

08198_20
['1/5/2022' '2/12/2022' '3/25/2022']
['1/5/2022' '2/12/2022']
      Sample  Date_HoaLy       DoAm   TroTong  TroKhongTan  Piperine  TinhDau
17  08198_20  2022-03-26  13.803595  3.566194          NaN  4.737441      3.2


08297_05
['1/11/2022' '2/15/2022' '4/18/2022' '11/9/2021' '11/12/2021' '11/15/2021']
['1/11/2022' '2/15/2022' '11/9/2021' '11/12/2021' '11/15/2021']
      Sample  Date_HoaLy       DoAm   TroTong  TroKhongTan  Piperine  TinhDau
31  08297_05  2022-04-18  12.254656  4.572334          NaN  5.710932      3.6


02422_04
['1/18/2022' '2/17/2022' '3/7/2022']
['1/18/2022' '2/17/2022' '3/7/2022']
       Sample  Date_HoaLy       DoAm   TroTong  TroKhongTan  Piperine  TinhDau
27   02422_04  2022-01-15  14.714533  3.937817          NaN  5.224666      3.8
158  02422_04  2022-03-09  14.146467  3.712632          NaN  4.902765      3.2


02563_02
['1/18/2022' '2/12/2022' '2/15/2022' '2/28/2022' '3/7/2022']
['1/18/2022' '2/12/2022' '2/15/2022' '2/28/2022' '3/7/2022']
       Samp

In [11]:
df[pd.isnull(df['Peperine'])]['Code2'].unique()

array(['08198_20', '08198_01', '08216_02', '08297_05', '02422_04',
       '02563_02', '02455_03', '02485_05', '02506_01', '08198_19',
       '08299_13', '08299_14', '08299_15', '08299_16', '00389_01',
       '08299_12', '00425_02', '00506_05', '08298_06', '00898_23',
       '00871_01', '00373_08', '00401_16', '06925_15', '00401_22',
       '00401_17', '00401_18', '06925_13', '02411_06', '02417_10',
       '02469_32', '00423_04', '00523_01', '00534_03', '08198_02',
       '08198_05', '08216_01', '08216_03', '08297_04', '08297_06',
       '08299_07', '08299_08', '08299_09', '08299_10', '08299_11'],
      dtype=object)

In [ ]:
arr_code_NIR = df['Code2'].unique()
arr_code_hoaly = df_hoaly['Sample'].unique()

In [14]:
df_hoaly_piperine['Sample'].unique()

array(['00425_02', '00423_04', '00506_05', '08216_01', '08299_07',
       '08299_08', '08299_09', '08299_10', '08299_11', '08299_12',
       '08299_13', '08299_14', '08299_15', '08299_16', '08201_17',
       '08198_18', '08198_19', '08198_20', '00389_01', '00398_02',
       '00389_03', '00389_04', '00389_05', '00389_06', '02506_01',
       '02563_02', '02455_03', '02422_04', '02485_05', '08216_03',
       '08297_04', '08297_05', '08297_06', '08198_02', '08198_05',
       '08198_04', '00855_10', '00856_14', '00141_28', '00110_11',
       '00110_13', '03682_17', '00104_19', '00098_20', '00893_35',
       '03682_18', '00507_21', '07338_18', '00108_05', '00108_04',
       '00850_04', '00109_02', '00893_24', '00213_10', '00110_12',
       '00850_11', '00871_02', '00110_06', '00855_09', '00843_13',
       '00854_15', '07338_19', '00507_20', '00072_22', '00141_27',
       '00141_29', '00825_30', '00142_31', '00107_14', '00847_33',
       '00893_34', '00893_36', '00867_03', '00373_05', '00373_

In [15]:
df_hoaly_piperine = df_hoaly[pd.notnull(df_hoaly['Peperine'])]


In [138]:
df_hoaly_piperine[df_hoaly_piperine['Sample'] == '08297_05']

,Sample,Date_HoaLy,DoAm,TroTong,Peperine,TinhDau


In [136]:
df[df['Code2'] == '08198_20']['extract_date_from_filename'].unique()

array([datetime.date(2022, 1, 5), datetime.date(2022, 2, 12),
       datetime.date(2022, 3, 25)], dtype=object)